In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 6.5MB 55kB/s eta 0:00:011
     |████████████████████████████████| 296kB 55.6MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp37-cp37m-linux_x86_64.whl size=1708029 sha256=0285dadac61f75d44844846a64b1b2b02148effb89d437ad77733aaa4bf19cd4
  Stored in directory: /home/aistudio/.cache/pip/wheels/b6/4f/e5/76b1296589466ff055f1e5b2d1fad86c6b33dc279553104896
Successfully built scikit-surprise


### **作业说明：由于colab上不去，改用百度AI studio来做**  
为了快点收敛学习率设置为0.1,没想到svd++的速度如此慢...

In [5]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold, split
from surprise import SVD, SVDpp


# 读取数据
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# 定义3个算法
# surprise 的svd应该就是 biased svd, 设置为false后为funk svd
# 跑 svd++ 的速度太慢，学习率调整为0.1
algo_LFM = SVD(biased=False, n_epochs=5, lr_all=0.1)  
algo_biased = SVD(n_epochs=5, lr_all=0.1)
algo_pp = SVDpp(n_epochs=5, lr_all=0.1)


# 跑模型
print("以下为 LFM 即 Funk SVD的结果： ")
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo_LFM.fit(trainset)
    predictions = algo_LFM.test(testset)
    accuracy.rmse(predictions, verbose=True)

print("以下为 Biased SVD的结果： ")
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo_biased.fit(trainset)
    predictions = algo_biased.test(testset)
    accuracy.rmse(predictions, verbose=True)
    
print("以下为 SVD++ 的结果： ")
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo_pp.fit(trainset)
    predictions = algo_pp.test(testset)
    accuracy.rmse(predictions, verbose=True)

# 预测
uid = str(196)
iid = str(592)
pred = algo_LFM.predict(uid, iid, r_ui=4, verbose=True)
pred = algo_biased.predict(uid, iid, r_ui=4, verbose=True)
pred = algo_pp.predict(uid, iid, r_ui=4, verbose=True)

以下为 LFM 即 Funk SVD的结果： 
RMSE: 1.1861
RMSE: 1.1731
RMSE: 1.1798
以下为 Biased SVD的结果： 
RMSE: 0.9543
RMSE: 0.9562
RMSE: 0.9557
以下为 SVD++ 的结果： 
RMSE: 0.9383
RMSE: 0.9394
RMSE: 0.9429
user: 196        item: 592        r_ui = 4.00   est = 2.88   {'was_impossible': False}
user: 196        item: 592        r_ui = 4.00   est = 3.73   {'was_impossible': False}
user: 196        item: 592        r_ui = 4.00   est = 3.49   {'was_impossible': False}


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 